In [62]:
"""
	Autores:
	- Junior Lara
	- Astrid Alvarado
"""

from pyspark.sql import SparkSession
from pyspark.sql.types import StringType
import pycountry

In [63]:
spark = SparkSession\
    .builder\
    .appName("Solver problema 1 - SQL")\
    .getOrCreate()

In [64]:
# ITEM 1: Ver cuáles países son los que tienen más usuarios que ven anime.
# Archivo necesiario: users-details-2023.csv
# Campos necesarios:
# 	- Mal ID
# 	- Location


# Direccionio del archivo
# Ej. file = "../../tests/vshort-users-details-2023.csv"
file = "../../tests/vshort-users-details-2023.csv"

users_details_2023 = spark\
	.read\
	.format("csv")\
	.option("header", "true")\
	.load(file)

users_details_2023.show(20)

+------+---------------+------+--------------------+--------------------+--------------------+------------+----------+--------+---------+-------+-------+-------------+-------------+---------+----------------+
|Mal ID|       Username|Gender|            Birthday|            Location|              Joined|Days Watched|Mean Score|Watching|Completed|On Hold|Dropped|Plan to Watch|Total Entries|Rewatched|Episodes Watched|
+------+---------------+------+--------------------+--------------------+--------------------+------------+----------+--------+---------+-------+-------+-------------+-------------+---------+----------------+
|     1|          Xinil|  Male|1985-03-04T00:00:...|          California|2004-11-05T00:00:...|       142.3|      7.37|     1.0|    233.0|    8.0|   93.0|         64.0|        399.0|     60.0|          8458.0|
|     3|        Aokaado|  Male|                NULL|        Oslo, Norway|2004-11-11T00:00:...|        68.6|      7.34|    23.0|    137.0|   99.0|   44.0|         40

In [65]:
# Se genera la VIEW
users_details_2023.createOrReplaceTempView("users_details_2023")

# Extraemos el campo Location y Mal ID
id_location = spark.sql("SELECT `Mal ID`, Location FROM users_details_2023")

In [66]:
# Mapeo manual de abreviaciones de subdivisiones a países
ABBREVIATION_MAP = {
    "ab": "Canada",
    "bc": "Canada",
    "on": "Canada",
    "qc": "Canada",
    "ny": "United States",
    "ca": "United States",
    "tx": "United States",
    "usa": "United States",
    "uk": "United Kingdom",
    "uae": "United Arab Emirates",
    "prc": "China",
    "us": "United States",
    "gb": "United Kingdom",
    "ru": "Russia",
    "au": "Australia",
    "nz": "New Zealand",
    "de": "Germany",
    "fr": "France",
    "es": "Spain",
    "it": "Italy",
    "jp": "Japan",
    "cn": "China",
    "kr": "South Korea",
    "br": "Brazil",
    "mx": "Mexico",
    "in": "India",
    "za": "South Africa",
    "se": "Sweden",
    "no": "Norway",
    "fi": "Finland",
    "dk": "Denmark",
    "ch": "Switzerland",
    "nl": "Netherlands",
    "be": "Belgium",
    "at": "Austria",
    "pl": "Poland",
    "ar": "Argentina",
    "cl": "Chile",
    "co": "Colombia",
    "ve": "Venezuela",
    "pe": "Peru",
    "ph": "Philippines",
    "id": "Indonesia",
    "my": "Malaysia",
    "sg": "Singapore",
    "th": "Thailand",
    "vn": "Vietnam",
    "sa": "Saudi Arabia",
    "eg": "Egypt",
    "tr": "Turkey",
    "gr": "Greece",
    "pt": "Portugal",
    "cz": "Czech Republic",
    "sk": "Slovakia",
    "hu": "Hungary",
    "ro": "Romania",
    "bg": "Bulgaria",
    "rs": "Serbia",
    "hr": "Croatia",
    "si": "Slovenia",
    "ua": "Ukraine",
    "by": "Belarus",
    "lt": "Lithuania",
    "lv": "Latvia",
    "ee": "Estonia",
    "is": "Iceland",
    "ie": "Ireland",
    "pk": "Pakistan",
    "bd": "Bangladesh",
    "lk": "Sri Lanka",
    "np": "Nepal",
    "af": "Afghanistan",
    "ir": "Iran",
    "iq": "Iraq",
    "sy": "Syria",
    "jo": "Jordan",
    "lb": "Lebanon",
    "il": "Israel",
    "ps": "Palestine",
    "kw": "Kuwait",
    "qa": "Qatar",
    "bh": "Bahrain",
    "om": "Oman",
    "ye": "Yemen",
    "dz": "Algeria",
    "ma": "Morocco",
    "tn": "Tunisia",
    "ly": "Libya",
    "sd": "Sudan",
    "ng": "Nigeria",
    "gh": "Ghana",
    "ke": "Kenya",
    "tz": "Tanzania",
    "ug": "Uganda",
    "zm": "Zambia",
    "zw": "Zimbabwe",
    "bw": "Botswana",
    "na": "Namibia",
    "ao": "Angola",
    "mz": "Mozambique",
    "mg": "Madagascar",
    "et": "Ethiopia",
    "sn": "Senegal",
    "ci": "Ivory Coast",
    "cm": "Cameroon",
    "cd": "Congo",
    "cg": "Republic Of The Congo",
    "ga": "Gabon",
    "gq": "Equatorial Guinea",
    "cv": "Cape Verde",
    "st": "Sao Tome And Principe",
    "sc": "Seychelles",
    "mu": "Mauritius",
    "km": "Comoros",
    "dj": "Djibouti",
    "er": "Eritrea",
    "so": "Somalia",
    "rw": "Rwanda",
    "bi": "Burundi",
    "mw": "Malawi",
    "ls": "Lesotho",
    "sz": "Eswatini",
    "sl": "Sierra Leone",
    "lr": "Liberia",
    "gm": "Gambia",
    "gw": "Guinea-Bissau",
    "gn": "Guinea",
    "ml": "Mali",
    "bf": "Burkina Faso",
    "ne": "Niger",
    "td": "Chad",
    "mr": "Mauritania",
    "bj": "Benin",
    "tg": "Togo",
    "cf": "Central African Republic",
    "ss": "South Sudan",
    "bt": "Bhutan",
    "mv": "Maldives",
    "kh": "Cambodia",
    "la": "Laos",
    "mm": "Myanmar",
    "bn": "Brunei",
    "tl": "Timor-Leste",
    "pg": "Papua New Guinea",
    "fj": "Fiji",
    "ws": "Samoa",
    "to": "Tonga",
    "vu": "Vanuatu",
    "sb": "Solomon Islands",
    "ki": "Kiribati",
    "tv": "Tuvalu",
    "nr": "Nauru",
    "pw": "Palau",
    "mh": "Marshall Islands",
    "fm": "Micronesia",
    "as": "American Samoa",
    "gu": "Guam",
    "mp": "Northern Mariana Islands",
    "ck": "Cook Islands",
    "nu": "Niue",
    "wf": "Wallis And Futuna",
    "pf": "French Polynesia",
    "nc": "New Caledonia",
    "tk": "Tokelau",
    "pn": "Pitcairn Islands",
    "gs": "South Georgia And The South Sandwich Islands",
    "sh": "Saint Helena",
    "fk": "Falkland Islands",
    "ai": "Anguilla",
    "bm": "Bermuda",
    "vg": "British Virgin Islands",
    "ky": "Cayman Islands",
    "ms": "Montserrat",
    "tc": "Turks And Caicos Islands",
    "vi": "US Virgin Islands",
    "pr": "Puerto Rico",
    "um": "United States Minor Outlying Islands",
    "hk": "Hong Kong",
    "mo": "Macau",
    "tw": "Taiwan",
    "fo": "Faroe Islands",
    "gl": "Greenland",
    "ax": "Aland Islands",
    "je": "Jersey",
    "gg": "Guernsey",
    "im": "Isle Of Man",
    "yt": "Mayotte",
    "re": "Reunion",
    "mq": "Martinique",
    "gp": "Guadeloupe",
    "bl": "Saint Barthelemy",
    "mf": "Saint Martin",
    "pm": "Saint Pierre And Miquelon",
    "tf": "French Southern Territories",
    "bv": "Bouvet Island",
    "hm": "Heard Island And Mcdonald Islands",
    "aq": "Antarctica",
    "cw": "Curacao",
    "sx": "Sint Maarten",
    "bq": "Caribbean Netherlands",
    "xk": "Kosovo"
}

# Función para identificar el país a partir de la ubicación.
def get_country(location):
    if not location or location.strip() == "":
        return "UNKNOWN"
    
    location = location.lower().strip()
    location_parts = [part.strip() for part in location.split(",")]  # Dividir por comas y limpiar espacios
    
    # Verificar si alguna parte coincide con una abreviación en el mapeo manual
    for part in location_parts:
        if part in ABBREVIATION_MAP:
            return ABBREVIATION_MAP[part]  # Devolver el país correspondiente
    
    # Verificar si el país ya está incluido en la ubicación
    for country in pycountry.countries:
        if country.name.lower() in location_parts or \
           country.alpha_2.lower() in location_parts or \
           country.alpha_3.lower() in location_parts:
            return country.name
    
    # Verificar subdivisiones (estados, provincias, etc.)
    for subdivision in pycountry.subdivisions:
        if subdivision.name.lower() in location_parts:
            return pycountry.countries.get(alpha_2=subdivision.country_code).name
    
    # Si no se encuentra, devolver "UNKNOWN"
    return "UNKNOWN"

# Registrar la función UDF en SQL
spark.udf.register("get_country_sql", get_country, StringType())

<function __main__.get_country(location)>

In [67]:
# Se genera la VIEW
id_location.createOrReplaceTempView("id_location")

# Mapear cada localización a un país por usuario.
id_country = spark.sql("""
    SELECT 
        `Mal ID` as user_id,
        get_country_sql(Location) AS Country
    FROM 
        id_location
""")

id_country.show(20)

+-------+-------------+
|user_id|      Country|
+-------+-------------+
|      1|United States|
|      3|       Norway|
|      4|    Australia|
|      9|      UNKNOWN|
|     18|      UNKNOWN|
|     20|       Norway|
|     23|       Canada|
|     36|      UNKNOWN|
|     44|      UNKNOWN|
|     47|      UNKNOWN|
|     53|      UNKNOWN|
|     66|       Canada|
|     70|      UNKNOWN|
|     71|      UNKNOWN|
|     77|      UNKNOWN|
|     80|      UNKNOWN|
|     82|       France|
|     83|United States|
|     90|      UNKNOWN|
|     91|       Canada|
+-------+-------------+
only showing top 20 rows



In [68]:
# Se genera la VIEW
id_country.createOrReplaceTempView("id_country")

# Consulta SQL para contar la cantidad de usuarios por país
user_count_by_country = spark.sql("""
    SELECT 
        Country,
        COUNT(user_id) AS user_count
    FROM 
        id_country
    GROUP BY 
        Country
    ORDER BY 
        user_count DESC
""")

# Mostrar los resultados
user_count_by_country.show(20)

+--------------+----------+
|       Country|user_count|
+--------------+----------+
|       UNKNOWN|       756|
| United States|        81|
|        Canada|        18|
|United Kingdom|        18|
|        Sweden|        13|
|        Israel|        10|
|       Finland|         9|
|        Norway|         7|
|       Germany|         6|
|        France|         6|
|     Lithuania|         5|
|        Mexico|         5|
|   Netherlands|         5|
|   Philippines|         4|
|        Greece|         4|
|       Georgia|         4|
|     Australia|         4|
|         Japan|         3|
|        Poland|         3|
|     Singapore|         2|
+--------------+----------+
only showing top 20 rows



In [69]:
# ITEM 2: Determinar si en dichos países las puntuaciones tienden a ser muy altas, bajas o equilibradas.
# Archivo necesiario: users-score-2023.csv
# Campos necesarios:
# 	- user_id
# 	- rating


# Direccionio del archivo
# Ej. file = "../../tests/vshort-users-score-2023.csv"
file = "../../tests/vshort-users-score-2023.csv"

users_score_2023 = spark\
	.read\
	.format("csv")\
	.option("header", "true")\
	.load(file)

users_score_2023.show(20)

+-------+--------+--------+--------------------+------+
|user_id|Username|anime_id|         Anime Title|rating|
+-------+--------+--------+--------------------+------+
|      1|   Xinil|      21|           One Piece|     9|
|      1|   Xinil|      48|         .hack//Sign|     7|
|      1|   Xinil|     320|              A Kite|     5|
|      1|   Xinil|      49|    Aa! Megami-sama!|     8|
|      1|   Xinil|     304|Aa! Megami-sama! ...|     8|
|      1|   Xinil|     306|Abenobashi Mahou☆...|     8|
|      1|   Xinil|      53|       Ai Yori Aoshi|     7|
|      1|   Xinil|      47|               Akira|     5|
|      1|   Xinil|     591|      Amaenaide yo!!|     6|
|      1|   Xinil|      54|   Appleseed (Movie)|     7|
|      1|   Xinil|      55|         Arc the Lad|     5|
|      1|   Xinil|      56|             Avenger|     6|
|      1|   Xinil|      57|                Beck|     9|
|      1|   Xinil|     368|         Bible Black|     5|
|      1|   Xinil|      68|      Black Cat (TV)|

In [70]:
# Se genera la VIEW
users_score_2023.createOrReplaceTempView("users_score_2023")

# Calcular el promedio (redondeado a 2 decimales) de las puntuaciones por usuario
average_ratings = spark.sql("""
    SELECT 
        user_id, 
        ROUND(AVG(rating), 2) AS average_rating
    FROM users_score_2023
    GROUP BY user_id
""")

average_ratings.show(20)

+-------+--------------+
|user_id|average_rating|
+-------+--------------+
|     23|          7.46|
|      9|          7.71|
|      1|          7.44|
|     20|          8.06|
|     37|           7.0|
|      4|          6.52|
+-------+--------------+



In [71]:
# Se generan las VIEWS
average_ratings.createOrReplaceTempView("average_ratings")
id_country.createOrReplaceTempView("id_country")

# JOIN para calcular el promedio por país y contar la cantidad de usuarios.
avg_ratings_by_country = spark.sql("""
    SELECT 
        c.Country,
        ROUND(AVG(a.average_rating), 2) AS average,
        COUNT(a.user_id) AS user_count
    FROM 
        average_ratings a
    INNER JOIN 
        id_country c
    ON 
        a.user_id = c.user_id
    GROUP BY 
        c.Country
""")

avg_ratings_by_country.show(20)

+-------------+-------+----------+
|      Country|average|user_count|
+-------------+-------+----------+
|United States|   7.44|         1|
|       Norway|   8.06|         1|
|      UNKNOWN|   7.71|         1|
|       Canada|   7.46|         1|
|    Australia|   6.52|         1|
+-------------+-------+----------+



In [72]:
# ITEM 3: Ver cuáles países tienen a las personas que dan muchas puntuaciones positivas y negativas.

# Se genera la VIEW
avg_ratings_by_country.createOrReplaceTempView("avg_ratings_by_country")

# Obtener el top 10 de países con más usuarios
top_10_countries = spark.sql("""
    SELECT *
    FROM avg_ratings_by_country
    ORDER BY user_count DESC
    LIMIT 10
""")

# Se genera la VIEW
top_10_countries.createOrReplaceTempView("top_10_countries")

# Obtener los 5 países con mejores promedios del top 10
top_5_best_averages = spark.sql("""
    SELECT *
    FROM top_10_countries
    ORDER BY average DESC
    LIMIT 5
""")

# Obtener los 5 países con peores promedios del top 10
top_5_worst_averages = spark.sql("""
    SELECT *
    FROM top_10_countries
    ORDER BY average ASC
    LIMIT 5
""")

# Mostrar los resultados
print("Top 5 países con mejores promedios:")
top_5_best_averages.show()

print("Top 5 países con peores promedios:")
top_5_worst_averages.show()

Top 5 países con mejores promedios:
+-------------+-------+----------+
|      Country|average|user_count|
+-------------+-------+----------+
|       Norway|   8.06|         1|
|      UNKNOWN|   7.71|         1|
|       Canada|   7.46|         1|
|United States|   7.44|         1|
|    Australia|   6.52|         1|
+-------------+-------+----------+

Top 5 países con peores promedios:


+-------------+-------+----------+
|      Country|average|user_count|
+-------------+-------+----------+
|    Australia|   6.52|         1|
|United States|   7.44|         1|
|       Canada|   7.46|         1|
|      UNKNOWN|   7.71|         1|
|       Norway|   8.06|         1|
+-------------+-------+----------+



In [73]:
spark.stop()